In [3]:
import torch
import torch.nn.functional as F
import allennlp

In [16]:

input = torch.randn([1,3,4]) # [batch, seq_len, num_classes]
input


tensor([[[-0.5880,  0.4605,  0.7157, -2.2909],
         [ 1.3694, -0.3685,  0.0109,  1.5163],
         [ 2.2856,  0.4966, -0.4558, -0.1334]]])

In [18]:
import numpy
targets = torch.LongTensor(numpy.random.randint(0,3,[1,3]))# [batch, seq_len]，其中的每个元素的取值是每个num_class指向的值
targets

tensor([[1, 0, 2]])

In [19]:
weights = torch.ones([2,3])
weights

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [20]:

from allennlp.nn.util import sequence_cross_entropy_with_logits

In [21]:
loss_sequence_cross = sequence_cross_entropy_with_logits(input, targets, weights, label_smoothing=0.1)
loss_sequence_cross

tensor(1.6762)

In [25]:
correct_loss = 0.0
for prediction, label in zip(input.squeeze(0), targets.squeeze(0)):
    print("prediction is:", prediction)
    print("label is:",label)
    prediction_log_softmax = torch.nn.functional.log_softmax(prediction, dim=-1)# 先执行softmax操作，然后再对结果取对数
#     print("prediction_log_softmax is:", prediction_log_softmax)
    correct_loss += prediction_log_softmax[label]*0.9
    correct_loss += prediction_log_softmax.sum() * 0.1/4
    
    print("\n\n")
    
correct_loss = -correct_loss / 3
correct_loss

prediction is: tensor([-0.5880,  0.4605,  0.7157, -2.2909])
label is: tensor(1)



prediction is: tensor([ 1.3694, -0.3685,  0.0109,  1.5163])
label is: tensor(0)



prediction is: tensor([ 2.2856,  0.4966, -0.4558, -0.1334])
label is: tensor(2)





tensor(1.6762)

所以调用sequence_cross_entropy_with_logits的loss_sequence_cross与我们计算的correct_loss是一样的。

这也说明计算交叉熵的过程就是对每一个label计算一个损失值【对一个token的所有的class的概率值先进行softmax，然后取对数，然后按照label_smooth的方法计算一个交叉熵损失值】。最后再对所有的损失值计算一个均值。

